<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Acknowledgements" data-toc-modified-id="Acknowledgements-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Acknowledgements</a></span></li><li><span><a href="#Prepare-data-and-model" data-toc-modified-id="Prepare-data-and-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare data and model</a></span></li><li><span><a href="#Make-feature-matrix-(word2vec,-votes,-stars)" data-toc-modified-id="Make-feature-matrix-(word2vec,-votes,-stars)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Make feature matrix (word2vec, votes, stars)</a></span></li><li><span><a href="#Create-Label-y-(Business-categories)" data-toc-modified-id="Create-Label-y-(Business-categories)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create Label y (Business categories)</a></span></li><li><span><a href="#Join-x,y-(feature-matrix,-category)-using-business_id" data-toc-modified-id="Join-x,y-(feature-matrix,-category)-using-business_id-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Join x,y (feature matrix, category) using business_id</a></span></li><li><span><a href="#Category-Prediction" data-toc-modified-id="Category-Prediction-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Category Prediction</a></span><ul class="toc-item"><li><span><a href="#Recall-(and-other-classification-metrics)" data-toc-modified-id="Recall-(and-other-classification-metrics)-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Recall (and other classification metrics)</a></span></li><li><span><a href="#RECOMMENDATIONS" data-toc-modified-id="RECOMMENDATIONS-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>RECOMMENDATIONS</a></span></li></ul></li></ul></div>

# Acknowledgements
Thanks to the tutorial: https://www.kaggle.com/c/word2vec-nlp-tutorial/overview/part-3-more-fun-with-word-vectors

# Prepare data and model

In [1]:
%matplotlib inline
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows=999
import numpy as np
import matplotlib.pyplot as plt

import re

import nltk
import nltk.data
nltk.download('stopwords')
from nltk.corpus import stopwords # Import the stop word list



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/daviderickson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def load_reviews(size='small'): 
    if size == 'small':
        filename = r'../../data/small-review.json'
    elif size == 'intermediate':
        filename = r'../../data/intermediate-review.json'
    elif size == 'full':
        filename = r'../../data/review.json'
    new_list = []
    for line in open(filename):
       new_list.append(json.loads(line))
    return pd.DataFrame.from_records(new_list)

dfreviews = load_reviews(size='intermediate')

In [3]:
dfreviews.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,ujmEBvifdJM6h6RLv4wQIg,0,2013-05-07 04:34:36,1,Q1sbwvVQXV2734tPgoKj4Q,1.0,Total bill for this horrible service? Over $8G...,6,hG7b0MtEbXx5QzbzE6C_VA
1,NZnhc2sEQy3RmzKTZnqtwQ,0,2017-01-14 21:30:33,0,GJXCdrto3ASJOqKeVWPi6Q,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,0,yXQM5uF2jS6es16SJzNHfg
2,WTqjgwHlXbSFevF32_DJVw,0,2016-11-09 20:09:03,0,2TzJjDVDEuAW6MR5Vuc1ug,5.0,I have to say that this office really has it t...,3,n6-Gk65cPZL6Uz8qRm3NYw
3,ikCg8xy5JIg_NGPx-MSIDA,0,2018-01-09 20:56:38,0,yi0R0Ugj_xUx_Nek0-_Qig,5.0,Went in for a lunch. Steak sandwich was delici...,0,dacAIZ6fTM6mqwW5uxkskg
4,b1b1eb3uo-w561D0ZfCEiQ,0,2018-01-30 23:07:38,0,11a8sVPMUFtaC7_ABRkmtw,1.0,Today was my second out of three sessions I ha...,7,ssoyf2_x0EQMed6fgHeMyQ


In [4]:
dfreviews.columns

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'useful', 'user_id'],
      dtype='object')

In [5]:
dfreviews['text'][0]

'Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.'

In [6]:
# For simplicity, drop anything that isn't a letter
# Numbers and symbols may have interesting meaning and could be explore later

def lettersOnly(string):
    return re.sub("[^a-zA-Z]", " ", string) 

dfreviews['text'] = dfreviews['text'].apply(lettersOnly)


In [7]:
dfreviews['text'][0]

'Total bill for this horrible service  Over   Gs  These crooks actually had the nerve to charge us     for   pills  I checked online the pills can be had for    cents EACH  Avoid Hospital ERs at all costs '

In [8]:
def review_to_wordlist(string, remove_stopwords=False):
    string = re.sub("[^a-zA-Z]", " ", string) # keep only letters. more complex model possible later
    words =  string.lower().split() # make everything lowercase. split into words
    if remove_stopwords:
        stops = set(stopwords.words('english')) # create a fast lookup for stopwords
        words = [w for w in words if not w in stops] # remove stopwords
    return( words) # return a list of words
    
# dfreviews['text'] = dfreviews['text'].apply(review_to_words) # apply to reviews in dataframe


In [9]:
# Word2Vec expects single sentences, each one as a list of words

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [10]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences")
for review in dfreviews["text"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences


In [11]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2020-01-22 12:49:05,613 : INFO : 'pattern' package not found; tag filters are not available for English
2020-01-22 12:49:05,631 : INFO : collecting all words and their counts
2020-01-22 12:49:05,632 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2020-01-22 12:49:05,845 : INFO : PROGRESS: at sentence #10000, processed 1088334 words, keeping 25539 word types
2020-01-22 12:49:06,032 : INFO : PROGRESS: at sentence #20000, processed 2172597 words, keeping 35463 word types
2020-01-22 12:49:06,229 : INFO : PROGRESS: at sentence #30000, processed 3251616 words, keeping 42649 word types
2020-01-22 12:49:06,588 : INFO : PROGRESS: at sentence #40000, processed 4373996 words, keeping 48893 word types
2020-01-22 12:49:06,865 : INFO : PROGRESS: at sentence #50000, processed 5471587 words, keeping 53964 word types
2020-01-22 12:49:07,115 : INFO : PROGRESS: at sentence #60000, processed 6570064 words, keeping 58362 word types
2020-01-22 12:49:07,313 : INFO : PROGRESS: at sentence #70000, processed 7667364 words, keeping 62704 word types
2020-01-22 12:49:07,518 : INFO : PROGRESS: at sentence #80000, processed 8768955 words, keeping 66443 word types
2020-01-22 12:49:07,721 : INFO : PROGRESS: at sentence #90000, processed 9872097 words, keeping 

2020-01-22 12:49:56,094 : INFO : EPOCH 3 - PROGRESS: at 77.19% examples, 393850 words/s, in_qsize 7, out_qsize 2
2020-01-22 12:49:57,095 : INFO : EPOCH 3 - PROGRESS: at 80.32% examples, 384762 words/s, in_qsize 7, out_qsize 0
2020-01-22 12:49:58,122 : INFO : EPOCH 3 - PROGRESS: at 85.05% examples, 383075 words/s, in_qsize 8, out_qsize 0
2020-01-22 12:49:59,142 : INFO : EPOCH 3 - PROGRESS: at 90.29% examples, 384375 words/s, in_qsize 7, out_qsize 0
2020-01-22 12:50:00,149 : INFO : EPOCH 3 - PROGRESS: at 93.99% examples, 379255 words/s, in_qsize 7, out_qsize 0
2020-01-22 12:50:01,153 : INFO : EPOCH 3 - PROGRESS: at 97.27% examples, 373284 words/s, in_qsize 7, out_qsize 0
2020-01-22 12:50:01,675 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-01-22 12:50:01,694 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-01-22 12:50:01,701 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-01-22 12:50:01,727 : INFO : worker thread fi

In [12]:
model.most_similar('pizza')

/Users/daviderickson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('pepperoni', 0.6951333284378052),
 ('crust', 0.6904908418655396),
 ('pizzas', 0.6607687473297119),
 ('calzone', 0.6115842461585999),
 ('margherita', 0.6036222577095032),
 ('lasagna', 0.5305042862892151),
 ('mozzarella', 0.5288822650909424),
 ('slice', 0.5260140299797058),
 ('subs', 0.5125235319137573),
 ('pasta', 0.5124855041503906)]

In [13]:
model.most_similar('service')

/Users/daviderickson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('waitstaff', 0.5213185548782349),
 ('staff', 0.47348546981811523),
 ('bartenders', 0.418529748916626),
 ('servers', 0.4099265933036804),
 ('communication', 0.40983614325523376),
 ('personality', 0.40979495644569397),
 ('hospitality', 0.39634326100349426),
 ('hostesses', 0.392345666885376),
 ('food', 0.38768860697746277),
 ('experience', 0.38350218534469604)]

In [14]:
model.most_similar('bad')

/Users/daviderickson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('terrible', 0.6226011514663696),
 ('horrible', 0.5923745036125183),
 ('good', 0.5514881610870361),
 ('awful', 0.5388085842132568),
 ('poor', 0.5210546255111694),
 ('disappointing', 0.500664234161377),
 ('subpar', 0.48986661434173584),
 ('alright', 0.48567622900009155),
 ('greatest', 0.46916308999061584),
 ('okay', 0.45262426137924194)]

In [15]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # WV.Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = int(0.)
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print ("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1
    return reviewFeatureVecs

In [16]:
# ****************************************************************
# Calculate average feature vectors
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_reviews = []
for review in dfreviews["text"]:
    clean_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

reviewDataVecs = getAvgFeatureVecs( clean_reviews, model, num_features )

Review 0 of 100000


/Users/daviderickson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review 1000 of 100000
Review 2000 of 100000
Review 3000 of 100000
Review 4000 of 100000
Review 5000 of 100000
Review 6000 of 100000
Review 7000 of 100000
Review 8000 of 100000


/Users/daviderickson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide


Review 9000 of 100000
Review 10000 of 100000
Review 11000 of 100000
Review 12000 of 100000
Review 13000 of 100000
Review 14000 of 100000
Review 15000 of 100000
Review 16000 of 100000
Review 17000 of 100000
Review 18000 of 100000
Review 19000 of 100000
Review 20000 of 100000
Review 21000 of 100000
Review 22000 of 100000
Review 23000 of 100000
Review 24000 of 100000
Review 25000 of 100000
Review 26000 of 100000
Review 27000 of 100000
Review 28000 of 100000
Review 29000 of 100000
Review 30000 of 100000
Review 31000 of 100000
Review 32000 of 100000
Review 33000 of 100000
Review 34000 of 100000
Review 35000 of 100000
Review 36000 of 100000
Review 37000 of 100000
Review 38000 of 100000
Review 39000 of 100000
Review 40000 of 100000
Review 41000 of 100000
Review 42000 of 100000
Review 43000 of 100000
Review 44000 of 100000
Review 45000 of 100000
Review 46000 of 100000
Review 47000 of 100000
Review 48000 of 100000
Review 49000 of 100000
Review 50000 of 100000
Review 51000 of 100000
Review 52000

# Make feature matrix (word2vec, votes, stars)

In [17]:
reviewDataVecs.shape[1]

300

In [18]:
# Add non-text data back to feature matrix
review_features = ['cool', 'funny', 'useful', 'stars' , 'business_id']
all_features_labels = ['w2v{}'.format(idx) for idx in range(reviewDataVecs.shape[1])] + review_features
all_features = np.append(reviewDataVecs, dfreviews[review_features].to_numpy(), 1)


In [19]:
# Create df 
all_features_df = pd.DataFrame(data=all_features, columns=all_features_labels)

# Convert all but business_id to numerical
business_ids = all_features_df['business_id']
all_features_df = all_features_df.iloc[:,:-1].astype('float64')
all_features_df['business_id'] = business_ids
del business_ids

# Group by business_id
all_features_business = all_features_df.groupby(by='business_id').mean()

In [20]:
all_features_business.head()

,w2v0,w2v1,w2v2,w2v3,w2v4,w2v5,w2v6,w2v7,w2v8,w2v9,w2v10,w2v11,w2v12,w2v13,w2v14,w2v15,w2v16,w2v17,w2v18,w2v19,w2v20,w2v21,w2v22,w2v23,w2v24,w2v25,w2v26,w2v27,w2v28,w2v29,w2v30,w2v31,w2v32,w2v33,w2v34,w2v35,w2v36,w2v37,w2v38,w2v39,w2v40,w2v41,w2v42,w2v43,w2v44,w2v45,w2v46,w2v47,w2v48,w2v49,w2v50,w2v51,w2v52,w2v53,w2v54,w2v55,w2v56,w2v57,w2v58,w2v59,w2v60,w2v61,w2v62,w2v63,w2v64,w2v65,w2v66,w2v67,w2v68,w2v69,w2v70,w2v71,w2v72,w2v73,w2v74,w2v75,w2v76,w2v77,w2v78,w2v79,w2v80,w2v81,w2v82,w2v83,w2v84,w2v85,w2v86,w2v87,w2v88,w2v89,w2v90,w2v91,w2v92,w2v93,w2v94,w2v95,w2v96,w2v97,w2v98,w2v99,w2v100,w2v101,w2v102,w2v103,w2v104,w2v105,w2v106,w2v107,w2v108,w2v109,w2v110,w2v111,w2v112,w2v113,w2v114,w2v115,w2v116,w2v117,w2v118,w2v119,w2v120,w2v121,w2v122,w2v123,w2v124,w2v125,w2v126,w2v127,w2v128,w2v129,w2v130,w2v131,w2v132,w2v133,w2v134,w2v135,w2v136,w2v137,w2v138,w2v139,w2v140,w2v141,w2v142,w2v143,w2v144,w2v145,w2v146,w2v147,w2v148,w2v149,w2v150,w2v151,w2v152,w2v153,w2v154,w2v155,w2v156,w2v157,w2v158,w2v159,w2v160,w2v161,w2v162,w2v163,w2v164,w2v165,w2v166,w2v167,w2v168,w2v169,w2v170,w2v171,w2v172,w2v173,w2v174,w2v175,w2v176,w2v177,w2v178,w2v179,w2v180,w2v181,w2v182,w2v183,w2v184,w2v185,w2v186,w2v187,w2v188,w2v189,w2v190,w2v191,w2v192,w2v193,w2v194,w2v195,w2v196,w2v197,w2v198,w2v199,w2v200,w2v201,w2v202,w2v203,w2v204,w2v205,w2v206,w2v207,w2v208,w2v209,w2v210,w2v211,w2v212,w2v213,w2v214,w2v215,w2v216,w2v217,w2v218,w2v219,w2v220,w2v221,w2v222,w2v223,w2v224,w2v225,w2v226,w2v227,w2v228,w2v229,w2v230,w2v231,w2v232,w2v233,w2v234,w2v235,w2v236,w2v237,w2v238,w2v239,w2v240,w2v241,w2v242,w2v243,w2v244,w2v245,w2v246,w2v247,w2v248,w2v249,w2v250,w2v251,w2v252,w2v253,w2v254,w2v255,w2v256,w2v257,w2v258,w2v259,w2v260,w2v261,w2v262,w2v263,w2v264,w2v265,w2v266,w2v267,w2v268,w2v269,w2v270,w2v271,w2v272,w2v273,w2v274,w2v275,w2v276,w2v277,w2v278,w2v279,w2v280,w2v281,w2v282,w2v283,w2v284,w2v285,w2v286,w2v287,w2v288,w2v289,w2v290,w2v291,w2v292,w2v293,w2v294,w2v295,w2v296,w2v297,w2v298,w2v299,cool,funny,useful,stars
business_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
--I7YYLada0tSLkORTHb5Q,-0.007499,0.015870,-0.009091,-0.006852,-0.000165,0.024793,0.008644,-0.014792,0.009979,0.003942,0.004100,-0.031356,-0.010247,0.005387,0.011627,0.020621,0.015115,-0.008609,0.002743,-0.010083,-0.006879,-0.005206,0.009593,-0.012116,-0.008580,0.004146,0.017497,-0.010027,-0.002657,-0.003539,-0.003672,-0.003878,-0.019413,-0.004522,0.010478,-0.011138,-0.001520,0.001624,0.002736,-0.001654,0.014157,0.012092,-0.011507,0.000065,0.021490,0.012381,-0.010089,-0.000056,0.006849,0.012484,0.022499,0.007505,0.001161,0.006571,-0.011028,-0.008634,-0.003242,-0.005032,0.016984,-0.006729,-0.007712,0.016816,-0.009824,-0.013810,-0.018497,0.005960,-0.003890,0.005296,0.001982,0.005957,0.010111,0.011505,-0.009504,0.006016,0.000145,-0.001613,0.003529,-0.009751,-0.001159,0.001350,0.004055,-0.012402,-0.011015,0.001537,0.009277,-0.006952,0.018196,0.011549,-0.003539,0.003471,-0.001982,-0.006949,-0.004149,-0.001219,-0.018856,0.018914,-0.002559,0.020924,-0.022010,-0.016058,-0.019431,0.024031,0.010372,-0.024269,0.001566,0.003882,-0.016965,0.014903,-0.000333,0.005388,-0.004297,-0.022004,0.007121,-0.015847,0.010955,0.008343,0.002458,-0.004447,-0.022745,-0.016351,-0.015083,0.016055,-0.010969,0.004302,-0.002894,-0.003404,0.013917,0.003703,-0.026776,-0.001713,0.000421,-0.011638,0.009560,0.006620,0.002412,0.005148,-0.014819,-0.002384,0.013313,-0.011222,-0.011357,-0.015396,-0.004863,-0.004180,-0.007797,-0.008371,0.005455,0.006528,0.016183,0.014788,0.010562,0.015067,0.008007,0.003813,-0.009594,-0.019872,0.006658,-0.004194,-0.000726,-0.019882,0.003207,0.008335,-0.003530,-0.008520,0.003569,0.014994,0.012201,0.009086,0.006450,-0.018860,-0.002093,0.004624,0.013989,-0

In [21]:
all_features_business.describe()

,w2v0,w2v1,w2v2,w2v3,w2v4,w2v5,w2v6,w2v7,w2v8,w2v9,w2v10,w2v11,w2v12,w2v13,w2v14,w2v15,w2v16,w2v17,w2v18,w2v19,w2v20,w2v21,w2v22,w2v23,w2v24,w2v25,w2v26,w2v27,w2v28,w2v29,w2v30,w2v31,w2v32,w2v33,w2v34,w2v35,w2v36,w2v37,w2v38,w2v39,w2v40,w2v41,w2v42,w2v43,w2v44,w2v45,w2v46,w2v47,w2v48,w2v49,w2v50,w2v51,w2v52,w2v53,w2v54,w2v55,w2v56,w2v57,w2v58,w2v59,w2v60,w2v61,w2v62,w2v63,w2v64,w2v65,w2v66,w2v67,w2v68,w2v69,w2v70,w2v71,w2v72,w2v73,w2v74,w2v75,w2v76,w2v77,w2v78,w2v79,w2v80,w2v81,w2v82,w2v83,w2v84,w2v85,w2v86,w2v87,w2v88,w2v89,w2v90,w2v91,w2v92,w2v93,w2v94,w2v95,w2v96,w2v97,w2v98,w2v99,w2v100,w2v101,w2v102,w2v103,w2v104,w2v105,w2v106,w2v107,w2v108,w2v109,w2v110,w2v111,w2v112,w2v113,w2v114,w2v115,w2v116,w2v117,w2v118,w2v119,w2v120,w2v121,w2v122,w2v123,w2v124,w2v125,w2v126,w2v127,w2v128,w2v129,w2v130,w2v131,w2v132,w2v133,w2v134,w2v135,w2v136,w2v137,w2v138,w2v139,w2v140,w2v141,w2v142,w2v143,w2v144,w2v145,w2v146,w2v147,w2v148,w2v149,w2v150,w2v151,w2v152,w2v153,w2v154,w2v155,w2v156,w2v157,w2v158,w2v159,w2v160,w2v161,w2v162,w2v163,w2v164,w2v165,w2v166,w2v167,w2v168,w2v169,w2v170,w2v171,w2v172,w2v173,w2v174,w2v175,w2v176,w2v177,w2v178,w2v179,w2v180,w2v181,w2v182,w2v183,w2v184,w2v185,w2v186,w2v187,w2v188,w2v189,w2v190,w2v191,w2v192,w2v193,w2v194,w2v195,w2v196,w2v197,w2v198,w2v199,w2v200,w2v201,w2v202,w2v203,w2v204,w2v205,w2v206,w2v207,w2v208,w2v209,w2v210,w2v211,w2v212,w2v213,w2v214,w2v215,w2v216,w2v217,w2v218,w2v219,w2v220,w2v221,w2v222,w2v223,w2v224,w2v225,w2v226,w2v227,w2v228,w2v229,w2v230,w2v231,w2v232,w2v233,w2v234,w2v235,w2v236,w2v237,w2v238,w2v239,w2v240,w2v241,w2v242,w2v243,w2v244,w2v245,w2v246,w2v247,w2v248,w2v249,w2v250,w2v251,w2v252,w2v253,w2v254,w2v255,w2v256,w2v257,w2v258,w2v259,w2v260,w2v261,w2v262,w2v263,w2v264,w2v265,w2v266,w2v267,w2v268,w2v269,w2v270,w2v271,w2v272,w2v273,w2v274,w2v275,w2v276,w2v277,w2v278,w2v279,w2v280,w2v281,w2v282,w2v283,w2v284,w2v285,w2v286,w2v287,w2v288,w2v289,w2v290,w2v291,w2v292,w2v293,w2v294,w2v295,w2v296,w2v297,w2v298,w2v299,cool,funny,useful,stars
count,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,139

# Create Label y (Business categories)

In [22]:
def load_business_df(): 
    filename = r'../../data/business.json'
    new_list = []
    for line in open(filename):
       new_list.append(json.loads(line))
    return pd.DataFrame.from_records(new_list)

dfbusiness = load_business_df()

In [23]:
dfbusiness.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,{'GoodForKids': 'False'},1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,None,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
3,"15655 W Roosevelt St, Ste 237",None,xvX2CttrVhyG2z1dFg_0xw,"Insurance, Financial Services",Goodyear,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,33.455613,-112.395596,Farmers Insurance - Paul Lorenz,85338,3,5.0,AZ
4,"4209 Stuart Andrew Blvd, Ste F","{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...",HhyxOkGAM07SRYtlQ4wMFQ,"Plumbing, Shopping, Local Services, Home Servi...",Charlotte,"{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ...",1,35.190012,-80.887223,Queen City Plumbing,28217,4,4.0,NC


# Join x,y (feature matrix, category) using business_id

In [24]:
dfbusiness.columns

Index(['address', 'attributes', 'business_id', 'categories', 'city', 'hours',
       'is_open', 'latitude', 'longitude', 'name', 'postal_code',
       'review_count', 'stars', 'state'],
      dtype='object')

In [25]:
len(dfbusiness['stars'].unique())

9

In [26]:
# Add business details to features df
keep_cols = ['business_id', 'categories', 'review_count']
all_features_business = all_features_business.merge(dfbusiness[keep_cols], how='left', on='business_id') 

In [27]:
all_features_business.head()

,business_id,w2v0,w2v1,w2v2,w2v3,w2v4,w2v5,w2v6,w2v7,w2v8,w2v9,w2v10,w2v11,w2v12,w2v13,w2v14,w2v15,w2v16,w2v17,w2v18,w2v19,w2v20,w2v21,w2v22,w2v23,w2v24,w2v25,w2v26,w2v27,w2v28,w2v29,w2v30,w2v31,w2v32,w2v33,w2v34,w2v35,w2v36,w2v37,w2v38,w2v39,w2v40,w2v41,w2v42,w2v43,w2v44,w2v45,w2v46,w2v47,w2v48,w2v49,w2v50,w2v51,w2v52,w2v53,w2v54,w2v55,w2v56,w2v57,w2v58,w2v59,w2v60,w2v61,w2v62,w2v63,w2v64,w2v65,w2v66,w2v67,w2v68,w2v69,w2v70,w2v71,w2v72,w2v73,w2v74,w2v75,w2v76,w2v77,w2v78,w2v79,w2v80,w2v81,w2v82,w2v83,w2v84,w2v85,w2v86,w2v87,w2v88,w2v89,w2v90,w2v91,w2v92,w2v93,w2v94,w2v95,w2v96,w2v97,w2v98,w2v99,w2v100,w2v101,w2v102,w2v103,w2v104,w2v105,w2v106,w2v107,w2v108,w2v109,w2v110,w2v111,w2v112,w2v113,w2v114,w2v115,w2v116,w2v117,w2v118,w2v119,w2v120,w2v121,w2v122,w2v123,w2v124,w2v125,w2v126,w2v127,w2v128,w2v129,w2v130,w2v131,w2v132,w2v133,w2v134,w2v135,w2v136,w2v137,w2v138,w2v139,w2v140,w2v141,w2v142,w2v143,w2v144,w2v145,w2v146,w2v147,w2v148,w2v149,w2v150,w2v151,w2v152,w2v153,w2v154,w2v155,w2v156,w2v157,w2v158,w2v159,w2v160,w2v161,w2v162,w2v163,w2v164,w2v165,w2v166,w2v167,w2v168,w2v169,w2v170,w2v171,w2v172,w2v173,w2v174,w2v175,w2v176,w2v177,w2v178,w2v179,w2v180,w2v181,w2v182,w2v183,w2v184,w2v185,w2v186,w2v187,w2v188,w2v189,w2v190,w2v191,w2v192,w2v193,w2v194,w2v195,w2v196,w2v197,w2v198,w2v199,w2v200,w2v201,w2v202,w2v203,w2v204,w2v205,w2v206,w2v207,w2v208,w2v209,w2v210,w2v211,w2v212,w2v213,w2v214,w2v215,w2v216,w2v217,w2v218,w2v219,w2v220,w2v221,w2v222,w2v223,w2v224,w2v225,w2v226,w2v227,w2v228,w2v229,w2v230,w2v231,w2v232,w2v233,w2v234,w2v235,w2v236,w2v237,w2v238,w2v239,w2v240,w2v241,w2v242,w2v243,w2v244,w2v245,w2v246,w2v247,w2v248,w2v249,w2v250,w2v251,w2v252,w2v253,w2v254,w2v255,w2v256,w2v257,w2v258,w2v259,w2v260,w2v261,w2v262,w2v263,w2v264,w2v265,w2v266,w2v267,w2v268,w2v269,w2v270,w2v271,w2v272,w2v273,w2v274,w2v275,w2v276,w2v277,w2v278,w2v279,w2v280,w2v281,w2v282,w2v283,w2v284,w2v285,w2v286,w2v287,w2v288,w2v289,w2v290,w2v291,w2v292,w2v293,w2v294,w2v295,w2v296,w2v297,w2v298,w2v299,cool,funny,useful,stars,categories,review_count
0,--I7YYLada0tSLkORTHb5Q,-0.007499,0.015870,-0.009091,-0.006852,-0.000165,0.024793,0.008644,-0.014792,0.009979,0.003942,0.004100,-0.031356,-0.010247,0.005387,0.011627,0.020621,0.015115,-0.008609,0.002743,-0.010083,-0.006879,-0.005206,0.009593,-0.012116,-0.008580,0.004146,0.017497,-0.010027,-0.002657,-0.003539,-0.003672,-0.003878,-0.019413,-0.004522,0.010478,-0.011138,-0.001520,0.001624,0.002736,-0.001654,0.014157,0.012092,-0.011507,0.000065,0.021490,0.012381,-0.010089,-0.000056,0.006849,0.012484,0.022499,0.007505,0.001161,0.006571,-0.011028,-0.008634,-0.003242,-0.005032,0.016984,-0.006729,-0.007712,0.016816,-0.009824,-0.013810,-0.018497,0.005960,-0.003890,0.005296,0.001982,0.005957,0.010111,0.011505,-0.009504,0.006016,0.000145,-0.001613,0.003529,-0.009751,-0.001159,0.001350,0.004055,-0.012402,-0.011015,0.001537,0.009277,-0.006952,0.018196,0.011549,-0.003539,0.003471,-0.001982,-0.006949,-0.004149,-0.001219,-0.018856,0.018914,-0.002559,0.020924,-0.022010,-0.016058,-0.019431,0.024031,0.010372,-0.024269,0.001566,0.003882,-0.016965,0.014903,-0.000333,0.005388,-0.004297,-0.022004,0.007121,-0.015847,0.010955,0.008343,0.002458,-0.004447,-0.022745,-0.016351,-0.015083,0.016055,-0.010969,0.004302,-0.002894,-0.003404,0.013917,0.003703,-0.026776,-0.001713,0.000421,-0.011638,0.009560,0.006620,0.002412,0.005148,-0.014819,-0.002384,0.013313,-0.011222,-0.011357,-0.015396,-0.004863,-0.004180,-0.007797,-0.008371,0.005455,0.006528,0.016183,0.014788,0.010562,0.015067,0.008007,0.003813,-0.009594,-0.019872,0.006658,-0.004194,-0.000726,-0.019882,0.003207,0.008335,-0.003530,-0.008520,0.003569,0.014994,0.012201,0.009086,0.006450,-0.018860,-0.002093,0.004624,0.013989,-0.011511,-0.000781,0.017287,0.005218,0.033981,-0.004402,-0.001438,0.023224,-0.008902,0.007502,-0.032931,0.001991,0.006755,-0.008278,-0.004967,-0.016611,0.002860,-0.027153,-0.007170,0.009309,-0.020619,0.000266,-0.001966,0.027710,-0.024209,-0.001210,-0.015006,-0.012201,0.003239,0.

In [28]:
all_features_business['categories'][0]

'Nightlife, Sports Bars, Restaurants, Bars, American (Traditional)'

In [29]:
all_features_business.head()

,business_id,w2v0,w2v1,w2v2,w2v3,w2v4,w2v5,w2v6,w2v7,w2v8,w2v9,w2v10,w2v11,w2v12,w2v13,w2v14,w2v15,w2v16,w2v17,w2v18,w2v19,w2v20,w2v21,w2v22,w2v23,w2v24,w2v25,w2v26,w2v27,w2v28,w2v29,w2v30,w2v31,w2v32,w2v33,w2v34,w2v35,w2v36,w2v37,w2v38,w2v39,w2v40,w2v41,w2v42,w2v43,w2v44,w2v45,w2v46,w2v47,w2v48,w2v49,w2v50,w2v51,w2v52,w2v53,w2v54,w2v55,w2v56,w2v57,w2v58,w2v59,w2v60,w2v61,w2v62,w2v63,w2v64,w2v65,w2v66,w2v67,w2v68,w2v69,w2v70,w2v71,w2v72,w2v73,w2v74,w2v75,w2v76,w2v77,w2v78,w2v79,w2v80,w2v81,w2v82,w2v83,w2v84,w2v85,w2v86,w2v87,w2v88,w2v89,w2v90,w2v91,w2v92,w2v93,w2v94,w2v95,w2v96,w2v97,w2v98,w2v99,w2v100,w2v101,w2v102,w2v103,w2v104,w2v105,w2v106,w2v107,w2v108,w2v109,w2v110,w2v111,w2v112,w2v113,w2v114,w2v115,w2v116,w2v117,w2v118,w2v119,w2v120,w2v121,w2v122,w2v123,w2v124,w2v125,w2v126,w2v127,w2v128,w2v129,w2v130,w2v131,w2v132,w2v133,w2v134,w2v135,w2v136,w2v137,w2v138,w2v139,w2v140,w2v141,w2v142,w2v143,w2v144,w2v145,w2v146,w2v147,w2v148,w2v149,w2v150,w2v151,w2v152,w2v153,w2v154,w2v155,w2v156,w2v157,w2v158,w2v159,w2v160,w2v161,w2v162,w2v163,w2v164,w2v165,w2v166,w2v167,w2v168,w2v169,w2v170,w2v171,w2v172,w2v173,w2v174,w2v175,w2v176,w2v177,w2v178,w2v179,w2v180,w2v181,w2v182,w2v183,w2v184,w2v185,w2v186,w2v187,w2v188,w2v189,w2v190,w2v191,w2v192,w2v193,w2v194,w2v195,w2v196,w2v197,w2v198,w2v199,w2v200,w2v201,w2v202,w2v203,w2v204,w2v205,w2v206,w2v207,w2v208,w2v209,w2v210,w2v211,w2v212,w2v213,w2v214,w2v215,w2v216,w2v217,w2v218,w2v219,w2v220,w2v221,w2v222,w2v223,w2v224,w2v225,w2v226,w2v227,w2v228,w2v229,w2v230,w2v231,w2v232,w2v233,w2v234,w2v235,w2v236,w2v237,w2v238,w2v239,w2v240,w2v241,w2v242,w2v243,w2v244,w2v245,w2v246,w2v247,w2v248,w2v249,w2v250,w2v251,w2v252,w2v253,w2v254,w2v255,w2v256,w2v257,w2v258,w2v259,w2v260,w2v261,w2v262,w2v263,w2v264,w2v265,w2v266,w2v267,w2v268,w2v269,w2v270,w2v271,w2v272,w2v273,w2v274,w2v275,w2v276,w2v277,w2v278,w2v279,w2v280,w2v281,w2v282,w2v283,w2v284,w2v285,w2v286,w2v287,w2v288,w2v289,w2v290,w2v291,w2v292,w2v293,w2v294,w2v295,w2v296,w2v297,w2v298,w2v299,cool,funny,useful,stars,categories,review_count
0,--I7YYLada0tSLkORTHb5Q,-0.007499,0.015870,-0.009091,-0.006852,-0.000165,0.024793,0.008644,-0.014792,0.009979,0.003942,0.004100,-0.031356,-0.010247,0.005387,0.011627,0.020621,0.015115,-0.008609,0.002743,-0.010083,-0.006879,-0.005206,0.009593,-0.012116,-0.008580,0.004146,0.017497,-0.010027,-0.002657,-0.003539,-0.003672,-0.003878,-0.019413,-0.004522,0.010478,-0.011138,-0.001520,0.001624,0.002736,-0.001654,0.014157,0.012092,-0.011507,0.000065,0.021490,0.012381,-0.010089,-0.000056,0.006849,0.012484,0.022499,0.007505,0.001161,0.006571,-0.011028,-0.008634,-0.003242,-0.005032,0.016984,-0.006729,-0.007712,0.016816,-0.009824,-0.013810,-0.018497,0.005960,-0.003890,0.005296,0.001982,0.005957,0.010111,0.011505,-0.009504,0.006016,0.000145,-0.001613,0.003529,-0.009751,-0.001159,0.001350,0.004055,-0.012402,-0.011015,0.001537,0.009277,-0.006952,0.018196,0.011549,-0.003539,0.003471,-0.001982,-0.006949,-0.004149,-0.001219,-0.018856,0.018914,-0.002559,0.020924,-0.022010,-0.016058,-0.019431,0.024031,0.010372,-0.024269,0.001566,0.003882,-0.016965,0.014903,-0.000333,0.005388,-0.004297,-0.022004,0.007121,-0.015847,0.010955,0.008343,0.002458,-0.004447,-0.022745,-0.016351,-0.015083,0.016055,-0.010969,0.004302,-0.002894,-0.003404,0.013917,0.003703,-0.026776,-0.001713,0.000421,-0.011638,0.009560,0.006620,0.002412,0.005148,-0.014819,-0.002384,0.013313,-0.011222,-0.011357,-0.015396,-0.004863,-0.004180,-0.007797,-0.008371,0.005455,0.006528,0.016183,0.014788,0.010562,0.015067,0.008007,0.003813,-0.009594,-0.019872,0.006658,-0.004194,-0.000726,-0.019882,0.003207,0.008335,-0.003530,-0.008520,0.003569,0.014994,0.012201,0.009086,0.006450,-0.018860,-0.002093,0.004624,0.013989,-0.011511,-0.000781,0.017287,0.005218,0.033981,-0.004402,-0.001438,0.023224,-0.008902,0.007502,-0.032931,0.001991,0.006755,-0.008278,-0.004967,-0.016611,0.002860,-0.027153,-0.007170,0.009309,-0.020619,0.000266,-0.001966,0.027710,-0.024209,-0.001210,-0.015006,-0.012201,0.003239,0.

In [30]:
def stringDFColToBinaryCols(df, series_name):
    # Create list of all categories
    all_cats = []
    for string in df[series_name]:
        string = str(string)
        cats = string.strip().replace(' ', '').split(',')
        for cat in cats:
            if cat not in all_cats:
                all_cats.append(cat)
    # Make binary for each cat for each row
    for cat in all_cats:
        df[cat] = df[series_name].str.strip().str.replace(' ', '').str.contains(cat)
        # This technique will have some problems. 'Golf' may appear in non-Golf categories (ie 'Disc Golf')
        # Can be fixed with regular expressions: ',Golf,' OR 'BOF Golf,' OR ',Golf EOF'
    
    return df, all_cats
        
all_features_business, all_cats = stringDFColToBinaryCols(all_features_business, 'categories')

/Users/daviderickson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if sys.path[0] == '':


In [31]:
print(all_cats)

['Nightlife', 'SportsBars', 'Restaurants', 'Bars', 'American(Traditional)', 'Pizza', 'HairRemoval', 'NailTechnicians', 'Beauty&Spas', 'NailSalons', 'Waxing', 'DaySpas', 'Electronics', 'ProfessionalServices', 'LocalServices', 'ElectronicsRepair', 'Computers', 'Shopping', 'Automotive', 'AutoRepair', 'Chinese', 'EyelashService', 'TobaccoShops', 'VapeShops', 'CarDealers', 'UsedCarDealers', 'Dentists', 'GeneralDentistry', 'CosmeticDentists', 'PediatricDentists', 'Health&Medical', 'Tex-Mex', 'Mexican', 'Arts&Entertainment', 'Festivals', 'Food', 'FoodTrucks', 'FarmersMarket', 'Portuguese', 'Bakeries', 'ChickenShop', 'Barbeque', 'EventPlanning&Services', 'EventPhotography', 'Photographers', 'SessionPhotography', 'SkinCare', 'Antiques', 'IceCream&FrozenYogurt', 'Donuts', 'SpecialtyFood', 'WebDesign', 'GraphicDesign', 'Marketing', 'RecyclingCenter', 'Caterers', 'Southern', 'ComfortFood', 'Breakfast&Brunch', 'French', 'American(New)', 'Burgers', 'Sandwiches', 'Coffee&Tea', 'Brasseries', 'Gyms', '

In [32]:
print(
    len(all_features_business[all_features_business['Golf']==True]), 
    len(all_features_business[all_features_business['DiscGolf']==True]), 
)

61 1


In [33]:
print(all_features_business[all_features_business['DiscGolf']==True]['categories'].values)
print('Should not have a True value for Golf, but does. Problem to deal with in the future.')
print(all_features_business[all_features_business['DiscGolf']==True]['Golf'].values)

['Sporting Goods, Active Life, Bike Rentals, Disc Golf, Shopping']
Should not have a True value for Golf, but does. Problem to deal with in the future.
[True]


In [34]:
all_features_business.head()

,business_id,w2v0,w2v1,w2v2,w2v3,w2v4,w2v5,w2v6,w2v7,w2v8,w2v9,w2v10,w2v11,w2v12,w2v13,w2v14,w2v15,w2v16,w2v17,w2v18,w2v19,w2v20,w2v21,w2v22,w2v23,w2v24,w2v25,w2v26,w2v27,w2v28,w2v29,w2v30,w2v31,w2v32,w2v33,w2v34,w2v35,w2v36,w2v37,w2v38,w2v39,w2v40,w2v41,w2v42,w2v43,w2v44,w2v45,w2v46,w2v47,w2v48,w2v49,w2v50,w2v51,w2v52,w2v53,w2v54,w2v55,w2v56,w2v57,w2v58,w2v59,w2v60,w2v61,w2v62,w2v63,w2v64,w2v65,w2v66,w2v67,w2v68,w2v69,w2v70,w2v71,w2v72,w2v73,w2v74,w2v75,w2v76,w2v77,w2v78,w2v79,w2v80,w2v81,w2v82,w2v83,w2v84,w2v85,w2v86,w2v87,w2v88,w2v89,w2v90,w2v91,w2v92,w2v93,w2v94,w2v95,w2v96,w2v97,w2v98,w2v99,w2v100,w2v101,w2v102,w2v103,w2v104,w2v105,w2v106,w2v107,w2v108,w2v109,w2v110,w2v111,w2v112,w2v113,w2v114,w2v115,w2v116,w2v117,w2v118,w2v119,w2v120,w2v121,w2v122,w2v123,w2v124,w2v125,w2v126,w2v127,w2v128,w2v129,w2v130,w2v131,w2v132,w2v133,w2v134,w2v135,w2v136,w2v137,w2v138,w2v139,w2v140,w2v141,w2v142,w2v143,w2v144,w2v145,w2v146,w2v147,w2v148,w2v149,w2v150,w2v151,w2v152,w2v153,w2v154,w2v155,w2v156,w2v157,w2v158,w2v159,w2v160,w2v161,w2v162,w2v163,w2v164,w2v165,w2v166,w2v167,w2v168,w2v169,w2v170,w2v171,w2v172,w2v173,w2v174,w2v175,w2v176,w2v177,w2v178,w2v179,w2v180,w2v181,w2v182,w2v183,w2v184,w2v185,w2v186,w2v187,w2v188,w2v189,w2v190,w2v191,w2v192,w2v193,w2v194,w2v195,w2v196,w2v197,w2v198,w2v199,w2v200,w2v201,w2v202,w2v203,w2v204,w2v205,w2v206,w2v207,w2v208,w2v209,w2v210,w2v211,w2v212,w2v213,w2v214,w2v215,w2v216,w2v217,w2v218,w2v219,w2v220,w2v221,w2v222,w2v223,w2v224,w2v225,w2v226,w2v227,w2v228,w2v229,w2v230,w2v231,w2v232,w2v233,w2v234,w2v235,w2v236,w2v237,w2v238,w2v239,w2v240,w2v241,w2v242,w2v243,w2v244,w2v245,w2v246,w2v247,w2v248,w2v249,w2v250,w2v251,w2v252,w2v253,w2v254,w2v255,w2v256,w2v257,w2v258,w2v259,w2v260,w2v261,w2v262,w2v263,w2v264,w2v265,w2v266,w2v267,w2v268,w2v269,w2v270,w2v271,w2v272,w2v273,w2v274,w2v275,w2v276,w2v277,w2v278,w2v279,w2v280,w2v281,w2v282,w2v283,w2v284,w2v285,w2v286,w2v287,w2v288,w2v289,w2v290,w2v291,w2v292,w2v293,w2v294,w2v295,w2v296,w2v297,w2v298,w2v299,cool,funny,useful,stars,categories,review_count,Nightlife,SportsBars,Restaurants,Bars,American(Traditional),Pizza,HairRemoval,NailTechnicians,Beauty&Spas,NailSalons,Waxing,DaySpas,Electronics,ProfessionalServices,LocalServices,ElectronicsRepair,Computers,Shopping,Automotive,AutoRepair,Chinese,EyelashService,TobaccoShops,VapeShops,CarDealers,UsedCarDealers,Dentists,GeneralDentistry,CosmeticDentists,PediatricDentists,Health&Medical,Tex-Mex,Mexican,Arts&Entertainment,Festivals,Food,FoodTrucks,FarmersMarket,Portuguese,Bakeries,ChickenShop,Barbeque,EventPlanning&Services,EventPhotography,Photographers,SessionPhotography,SkinCare,Antiques,IceCream&FrozenYogurt,Donuts,SpecialtyFood,WebDesign,GraphicDesign,Marketing,RecyclingCenter,Caterers,Southern,ComfortFood,Breakfast&Brunch,French,American(New),Burgers,Sandwiches,Coffee&Tea,Brasseries,Gyms,ChildCare&DayCare,LeisureCenters,Fitness&Instruction,ActiveLife,HardwareStores,Home&Garden,RealEstate,Condominiums,Hotels,HomeServices,ShoppingCenters,Hotels&Travel,HairSalons,EthnicFood,Turkish,InternationalGrocery,TapasBars,ShippingCenters,PrintingServices,Massage,MassageTherapy,Reflexology,Buffets,Korean,SushiBars,Japanese,Cafes,Soup,Golf,Venues&EventSpaces,AutoDetailing,BodyShops,AutoCustomization,Towing,Trainers,WeightLossCenters,FoodDeliveryServices,FastFood,Delis,Ethiopian,Vegetarian,Painters,DrywallInstallation&Repair,StuccoServices,Orthodontists,Periodontists,OralSurgeons,Piercing,Tattoo,Chiropractors,Optometrists,Italian,Couriers&DeliveryServices,PublicServices&Government,SportingGoods,Fashion,GolfEquipment,Bikes,Ski&SnowboardShops,SportsWear,BikeRepair/Maintenance,Filipino,PetGroomers,Veterinarians,PetSitting,Pets,PetServices,AutoGlassServices,RealEstateServices,RealEstateAgents,Pakistani,Indian,CardioClasses,DanceStudios,ChickenWings,Cosmetics&BeautySupply,Desserts,Sewing&Alterations,Arts&Crafts,Wheel&RimRepair,Tires,AutoParts&Supplies,Colonics,Saunas,Doctors,MedicalSpas,Naturopathic/Holistic,MeditationCenters,Reiki,Spi

In [35]:
# Clean

# Remove rows with NaNs
print('Before: ', len(all_features_business))
all_features_business = all_features_business.dropna(axis=0)
print('After:  ', len(all_features_business))

Before:  13943
After:   13922


In [36]:
# First, shuffle the dataframe 
# and reset the index. (Makes for easier handling of train/test later)
all_features_business = all_features_business.sample(frac=1).reset_index(drop=True)

# Create final y and x 
y_df = all_features_business[all_cats]
x_cols = [ele for ele in all_features_business.columns if ele not in all_cats+['categories', 'business_id']]
# May also want to remove from x_cols: 'cool', 'funny', 'useful', 'stars', 'categories', 'review_count' 
# May also want to drop rows that do not contain more than a threshold number of reviews (20?, 100?)
x_df = all_features_business[x_cols]

# Numpy arrays
x = x_df.values
y = y_df.values

# Classifier wants 1/0, not T/F
y = y.astype(int)

# Split into Train/Test sets
def splitSets(x, y, test_size=0.2):
    test_size_absolute = np.int(test_size * len(x))
    X_test, X_train = x[:test_size_absolute,:], x[test_size_absolute:,:]
    y_test, y_train = y[:test_size_absolute,:], y[test_size_absolute:,:]
    return X_train, X_test, y_train, y_test
    
test_size = 0.2
X_train, X_test, y_train, y_test = splitSets(x, y, test_size=test_size)

In [37]:
y_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# Category Prediction

In [38]:
# Multilabel Classification
# RandomForestClassifier supports multilabel classification

# Most other classifiers will require use of 
    # sklearn.multioutput.MultiOutputClassifier to run a separate classifier model for each targe
    
from sklearn.ensemble import RandomForestClassifier

In [39]:
rfc = RandomForestClassifier(n_estimators=10, n_jobs=-1)

In [40]:
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

## Recall (and other classification metrics)

In our case we want a Recall = TPR (True Positive Rate) close to 1 since we want to Recall ALL correct categories. 

The only requirement we have for Precision is that it be less than 1. This is because we want some FPs (False Positives) since these are what WE ARE RECOMMENDING!!

In [41]:
from sklearn.metrics import classification_report

y_predict = rfc.predict(X_test)
print(classification_report(y_test, y_predict, target_names=all_cats))

                                precision    recall  f1-score   support

                     Nightlife       0.83      0.31      0.45       234
                    SportsBars       0.00      0.00      0.00        24
                   Restaurants       0.92      0.88      0.90       983
                          Bars       0.78      0.31      0.44       259
         American(Traditional)       0.00      0.00      0.00         0
                         Pizza       0.97      0.27      0.42       104
                   HairRemoval       0.25      0.03      0.06        59
               NailTechnicians       0.00      0.00      0.00         2
                   Beauty&Spas       0.97      0.60      0.75       250
                    NailSalons       0.94      0.47      0.62        64
                        Waxing       1.00      0.03      0.05        36
                       DaySpas       1.00      0.03      0.06        35
                   Electronics       1.00      0.04      0.07  

/Users/daviderickson/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/daviderickson/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/daviderickson/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [42]:
from sklearn.metrics import recall_score 

recall_all_cats = recall_score(y_test, y_predict, average=None)
recall_all_cats

/Users/daviderickson/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


array([0.31196581, 0.        , 0.87792472, ..., 0.        , 0.        ,
       0.        ])

## RECOMMENDATIONS

Look at the recommendations. Recommendations are the categories that are predicted, but not present in the data. 

In [43]:
y_proba = rfc.predict_proba(X_test)

In [44]:
print( len(y_proba), ' L')
print( len(y_proba[0]), ' W')
print( len(y_proba[0][0]), " D (0: False prob'y, 1: True prob'y)")

1090  L
2784  W
2  D (0: False prob'y, 1: True prob'y)


In [45]:
y_proba[0][0]

array([1., 0.])

In [46]:
reccs_binary = (y_test == 0) & (y_predict == 1)
reccs_binary

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [47]:
all_cats_ser = pd.Series(data=all_cats)

In [48]:
all_cats_true = []
all_cats_recc = []
for biz in range(len(y_test)):
    cats_true = ', '.join(list(all_cats_ser[y_test[biz,:]==1]))
    all_cats_true.append(cats_true)
    
    cats_recc = ', '.join(list(all_cats_ser[reccs_binary[biz,:]==True]))
    all_cats_recc.append(cats_recc)

reccs_df = pd.DataFrame(data=all_cats_true, columns=['Labeled'])
reccs_df['Recommended'] = all_cats_recc
reccs_df.tail()

,Labeled,Recommended
2779,"Health&Medical, Massage, MassageTherapy, Chiro...",
2780,"HomeServices, LandscapeArchitects, Landscaping...",
2781,"Restaurants, Food, Vegetarian, Vegan, Live/Raw...",
2782,"ActiveLife, Golf",
2783,"ProfessionalServices, HomeServices, InternetSe...",


In [49]:
list(all_features_business.columns)

['business_id',
 'w2v0',
 'w2v1',
 'w2v2',
 'w2v3',
 'w2v4',
 'w2v5',
 'w2v6',
 'w2v7',
 'w2v8',
 'w2v9',
 'w2v10',
 'w2v11',
 'w2v12',
 'w2v13',
 'w2v14',
 'w2v15',
 'w2v16',
 'w2v17',
 'w2v18',
 'w2v19',
 'w2v20',
 'w2v21',
 'w2v22',
 'w2v23',
 'w2v24',
 'w2v25',
 'w2v26',
 'w2v27',
 'w2v28',
 'w2v29',
 'w2v30',
 'w2v31',
 'w2v32',
 'w2v33',
 'w2v34',
 'w2v35',
 'w2v36',
 'w2v37',
 'w2v38',
 'w2v39',
 'w2v40',
 'w2v41',
 'w2v42',
 'w2v43',
 'w2v44',
 'w2v45',
 'w2v46',
 'w2v47',
 'w2v48',
 'w2v49',
 'w2v50',
 'w2v51',
 'w2v52',
 'w2v53',
 'w2v54',
 'w2v55',
 'w2v56',
 'w2v57',
 'w2v58',
 'w2v59',
 'w2v60',
 'w2v61',
 'w2v62',
 'w2v63',
 'w2v64',
 'w2v65',
 'w2v66',
 'w2v67',
 'w2v68',
 'w2v69',
 'w2v70',
 'w2v71',
 'w2v72',
 'w2v73',
 'w2v74',
 'w2v75',
 'w2v76',
 'w2v77',
 'w2v78',
 'w2v79',
 'w2v80',
 'w2v81',
 'w2v82',
 'w2v83',
 'w2v84',
 'w2v85',
 'w2v86',
 'w2v87',
 'w2v88',
 'w2v89',
 'w2v90',
 'w2v91',
 'w2v92',
 'w2v93',
 'w2v94',
 'w2v95',
 'w2v96',
 'w2v97',
 'w2v98',
 'w2

In [50]:
reccs_df['categories'] = all_features_business['categories'].iloc[:len(reccs_df)]
reccs_df['business_id'] = all_features_business['business_id'].iloc[:len(reccs_df)]
reccs_df.tail()

,Labeled,Recommended,categories,business_id
2779,"Health&Medical, Massage, MassageTherapy, Chiro...",,"Massage Therapy, Health & Medical, Chiropractors",dHLFyEOj3CMi-pvrQanzxw
2780,"HomeServices, LandscapeArchitects, Landscaping...",,"Landscaping, Landscape Architects, Patio Cover...",9l09oYjDG0IQtZUitbhEAA
2781,"Restaurants, Food, Vegetarian, Vegan, Live/Raw...",,"Restaurants, Vegan, Live/Raw Food, Vegetarian",R1jJQi2yR44D_2ileqr8kA
2782,"ActiveLife, Golf",,"Active Life, Golf",aIqmckwjedtsvMKwPOEhhw
2783,"ProfessionalServices, HomeServices, InternetSe...",,"Television Service Providers, Home Services, P...",9gKE3S1PB67Ls6tYtoGVKQ


In [51]:
# This is where I need to pick up. I need to match the dataframes 
# so that I can match reviews etc and judge how well the recommender is doing

In [52]:
list(all_features_business['categories'].tail())

['Medical Centers, Doctors, Obstetricians & Gynecologists, Health & Medical',
 'Restaurants, Chinese',
 'Brazilian, Restaurants, Steakhouses',
 'Beer Bar, Bars, Sports Bars, Nightlife',
 'Restaurants, Thai']

In [53]:
len(reccs_df[reccs_df['Recommended']!=''])

273

In [54]:
reccs_df[reccs_df['Recommended']!=''].sort_values(by='Recommended')

,Labeled,Recommended,categories,business_id
429,"Beauty&Spas, Tanning",ActiveLife,"Tanning, Beauty & Spas",Cn-tvkG09gaSWGif-lWNqw
757,"ReligiousOrganizations, Churches",Arts&Entertainment,"Religious Organizations, Churches",99HjFNBu3p69JvDV2cpQFw
1409,"EventPlanning&Services, Venues&EventSpaces, Pu...",Arts&Entertainment,"Landmarks & Historical Buildings, Venues & Eve...",70ThpWAeQV8PEyJXJNid4Q
360,"Automotive, CarDealers",AutoRepair,"Car Dealers, Automotive",4oPqNwH6oULYu_X_L4A5xw
257,"Automotive, BodyShops, Towing",AutoRepair,"Towing, Body Shops, Automotive",ML2t7AfdnHNf9df6CT27FA
955,"Automotive, Wheel&RimRepair, Tires, AutoParts&...",AutoRepair,"Automotive, Tires, Wheel & Rim Repair, Auto Pa...",nqgeTj6bfIMY0v2J-vZa8A
379,"Automotive, Tires","AutoRepair, OilChangeStations","Tires, Automotive",sDhwuuRKr7Phz_XDYSleAQ
2730,"HomeServices, Flooring, Carpeting, CarpetInsta...",Automotive,"Home Services, Siding, Carpeting, Flooring, Ca...",0ZplgTLfGZvIKkG63ohfog
2536,"Hotels, Hotels&Travel, CarRental",Automotive,"Car Rental, Hotels & Travel",CAzLsMtoCZAuBeLnnOqmxw
827,"EventPlanning&Services, Hotels, Hotels&Travel,...",Automotive,"Tours, Limos, Hotels & Travel, Party Bus Renta...",GIzHC2PUrWC6_50C8R0CUA


In [55]:
dfreviews.columns

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'useful', 'user_id'],
      dtype='object')

In [56]:
pd.set_option('display.max_colwidth',2000)
dfreviews[dfreviews['business_id']=='KN0gPRzDvA6uVYims2KA0w']['text']

6680                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 This cute little place is really nearby my house and they have a lot of nice comfort food here   I ve only ever gone here for brunch and their portions are definitely big enough to share   You can add little things  ie  side of

In [57]:
# Given X_test row, find identical row in all_features_business and use that to find 'business_id'
all_features_business[x_cols].values == X_test[0]

array([[ True,  True,  True, ...,  True,  True,  True],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [58]:
X_test.shape

(2784, 305)

In [59]:
all_features_business[x_cols].values.shape

(13922, 305)